In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import kerastuner as kt
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

# 1. Veriyi Yükleme ve Aylık Dönüşüm
def load_data():
    # Her mağazaya ait dosyaları okuyoruz 
    df1 = pd.read_excel('/content/data/magaza_A.xlsx')
    df2 = pd.read_excel('/content/data/magaza_B.xls')

    
    for df in [df1, df2]:
        df['Tarih'] = pd.to_datetime(df['Tarih'])
        df['Ay'] = df['Tarih'].dt.to_period('M')

    # Her dosya için aylık özet oluşturma
    def aylik_ozet(df):
        aylik_df = df.groupby(['Ay', 'DepoRef']).agg({
            'Ciro': 'mean',           # Ciro için ortalama
            'HaftaSonu': 'sum',       # HaftaSonu için toplam
            'BayramEtkisi': 'sum',    # BayramEtkisi için toplam
            'CalismaGunSayisi': 'sum' # CalismaGunSayisi için toplam
        }).reset_index()
        aylik_df['Ay'] = aylik_df['Ay'].dt.to_timestamp()
        return aylik_df

    df1 = aylik_ozet(df1)
    df2 = aylik_ozet(df2)

    return pd.concat([df1, df2], ignore_index=True)

# 2. Özellik Mühendisliği ve One-Hot Encoding
def feature_engineering(data):
    """
    Ay bilgisi için one-hot encoding ve diğer özellik dönüşümleri
    """
    # Tarih özelliklerini çıkarma
    data['Month'] = data['Ay'].dt.month
    data['Quarter'] = data['Ay'].dt.quarter
    data['Year'] = data['Ay'].dt.year
    
    # Ay için one-hot encoding (Dummy Variable Trap önlemek için drop_first=True)
    month_dummies = pd.get_dummies(data['Month'], prefix='Month', drop_first=True)
    data = pd.concat([data, month_dummies], axis=1)
    
    # Çeyrek için one-hot encoding (Dummy Variable Trap önlemek için drop_first=True)
    quarter_dummies = pd.get_dummies(data['Quarter'], prefix='Quarter', drop_first=True)
    data = pd.concat([data, quarter_dummies], axis=1)
    
    # Orijinal Month ve Quarter sütunlarını kaldır
    data = data.drop(['Month', 'Quarter'], axis=1)
    
    return data


# 3. Dinamik Özellik Seçimi ve Korelasyon Matrisi Gösterimi
def dinamik_oznitelik_seçimi(data, target_column='Ciro', threshold=0.1):
    """
    Korelasyon tabanlı özellik seçimi (one-hot encoded özellikler dahil)
    """
    # 'Ay' ve 'DepoRef' sütunlarını çıkarıyoruz
    exclude_cols = ['Ay', 'DepoRef']
    data_numeric = data.drop(columns=exclude_cols)
    
    correlation_matrix = data_numeric.corr()
    print("Korelasyon Matrisi:\n", correlation_matrix[target_column].round(3))
    
    ciro_correlations = correlation_matrix[target_column]
    # Ciro'nun kendisi hariç, eşik değerini geçen özellikleri seç
    selected_features = ciro_correlations[
        (abs(ciro_correlations) > threshold) & 
        (ciro_correlations.index != target_column)
    ].index.tolist()
    
    # Ciro'yu da ekle (target değişken)
    selected_features.append(target_column)
    
    print(f"Seçilen öznitelikler (eşik {threshold}):", selected_features)
    return selected_features, correlation_matrix

# 4. Çok Değişkenli Zaman Serisi Dataset Oluşturma
def create_multivariate_dataset(dataset, time_step=12):
    """
    Çok değişkenli zaman serisi için veri seti oluşturma
    dataset: (samples, features) şeklinde numpy array
    """
    X, y = [], []
    for i in range(len(dataset) - time_step):
        # X: önceki time_step adımının tüm özellikleri
        X.append(dataset[i:(i + time_step), :])
        # y: sadece Ciro değeri (ilk sütun)
        y.append(dataset[i + time_step, 0])  # Ciro ilk sütunda
    return np.array(X), np.array(y)

# 5. Model ve Hiperparametre Optimizasyonu
def build_model(hp, time_step, n_features):
    """
    Çok değişkenli LSTM modeli
    """
    model = Sequential()
    
    # LSTM katmanları
    model.add(LSTM(
        units=hp.Int('lstm_units_1', min_value=32, max_value=128, step=16),
        return_sequences=True,
        input_shape=(time_step, n_features),
        dropout=hp.Float('lstm_dropout_1', 0.1, 0.3, step=0.05),
        recurrent_dropout=hp.Float('lstm_recurrent_dropout_1', 0.1, 0.3, step=0.05)
    ))
    
    model.add(LSTM(
        units=hp.Int('lstm_units_2', min_value=16, max_value=64, step=8),
        return_sequences=False,
        dropout=hp.Float('lstm_dropout_2', 0.1, 0.3, step=0.05),
        recurrent_dropout=hp.Float('lstm_recurrent_dropout_2', 0.1, 0.3, step=0.05)
    ))
    
    # Dense katmanlar
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i}', min_value=16, max_value=64, step=8),
            activation='relu',
            kernel_regularizer=l2(hp.Float(f'l2_{i}', min_value=1e-4, max_value=1e-2, sampling='log'))
        ))
        model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.4, step=0.05)))
    
    # Çıktı katmanı
    model.add(Dense(1, activation='linear'))
    
    # Optimizer
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    optimizer = Adam(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# 6. Ana Eğitim ve Tahmin Fonksiyonu
def train_and_predict():
    data = load_data()
    
    # Özellik mühendisliği
    data = feature_engineering(data)
    
    time_step = 12
    store_results = {}
    
    # Early Stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )
    
    # Her mağaza için işlemleri gerçekleştirelim
    for store in data['DepoRef'].unique():
        print(f"\n{'='*50}")
        print(f"Mağaza {store} için model eğitiliyor...")
        print(f"{'='*50}")
        
        store_data = data[data['DepoRef'] == store].copy()
        
        # Dinamik özellik seçimi
        selected_features, corr_matrix = dinamik_oznitelik_seçimi(store_data)
        
        # Seçilen özellikleri al ve sırala (Ciro'yu başa koy)
        feature_cols = ['Ay'] + selected_features
        store_data = store_data[feature_cols]
        
        # Ciro'yu ilk sütuna taşı
        cols = store_data.columns.tolist()
        cols.remove('Ciro')
        cols.remove('Ay')
        store_data = store_data[['Ay', 'Ciro'] + cols]
        
        # Tüm verileri tarih sırasına göre sıralama
        store_data = store_data.sort_values('Ay')
        
        print(f"Kullanılan özellikler: {store_data.columns.tolist()[1:]}")  # Ay hariç
        
        # Gerçek değerler
        actual_values = {}
        for month in [1, 2]:
            month_data = store_data[(store_data['Ay'].dt.year == 2025) & (store_data['Ay'].dt.month == month)]
            if not month_data.empty:
                actual_values[month] = month_data['Ciro'].values[0]
        
        # AŞAMA 1: 2023-2024 verilerini kullanarak 2025 Ocak ayını tahmin et
        print("\n1. AŞAMA: 2023-2024 verileriyle eğitim, 2025 Ocak tahmini")
        
        train_data_stage1 = store_data[store_data['Ay'].dt.year.isin([2023, 2024])].copy()
        train_features_stage1 = train_data_stage1.drop(['Ay'], axis=1).values
        
        # Normalizasyon (tüm özellikler için)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_train_stage1 = scaler.fit_transform(train_features_stage1)
        
        # Veri seti oluşturma
        X_train_stage1, y_train_stage1 = create_multivariate_dataset(scaled_train_stage1, time_step)
        
        if len(X_train_stage1) == 0:
            print(f"Mağaza {store} için yeterli veri yok!")
            continue
        
        print(f"Eğitim veri şekli: X: {X_train_stage1.shape}, y: {y_train_stage1.shape}")
        
        # Hiperparametre optimizasyonu
        tuner_stage1 = kt.RandomSearch(
            lambda hp: build_model(hp, time_step, scaled_train_stage1.shape[1]),
            objective='val_loss',
            max_trials=5,  # Hızlı test için azaltıldı
            executions_per_trial=1,
            directory='kt_dir',
            project_name=f'store_{store}_stage1'
        )
        
        tuner_stage1.search(
            X_train_stage1, y_train_stage1, 
            epochs=50, 
            batch_size=min(8, len(X_train_stage1)//2),
            validation_split=0.2, 
            verbose=0, 
            callbacks=[early_stopping]
        )
        
        # En iyi modeli al ve tahmin yap
        best_model_stage1 = tuner_stage1.get_best_models(num_models=1)[0]
        
        # Ocak 2025 tahmini
        input_jan_2025 = scaled_train_stage1[-time_step:].reshape(1, time_step, -1)
        jan_2025_pred_scaled = best_model_stage1.predict(input_jan_2025, verbose=0)
        
        # Tahmin sonucunu ters normalize et (sadece Ciro için)
        dummy_array = np.zeros((1, scaled_train_stage1.shape[1]))
        dummy_array[0, 0] = jan_2025_pred_scaled[0, 0]
        jan_2025_pred = scaler.inverse_transform(dummy_array)[0, 0]
        
        # Sonuçları kaydet
        predictions = {'jan': jan_2025_pred}
        
        print(f"2025 Ocak Tahmini: {jan_2025_pred:.2f} TL")
        if 1 in actual_values:
            print(f"2025 Ocak Gerçek: {actual_values[1]:.2f} TL")
            print(f"Ocak Hatası: {abs(jan_2025_pred - actual_values[1]):.2f} TL")
        
        # Mağaza sonuçlarını kaydet
        store_results[store] = {
            'predictions': predictions,
            'actual_values': actual_values,
            'selected_features': selected_features,
            'correlation_matrix': corr_matrix,
            'train_data': train_data_stage1
        }
    
    return store_results

# 7. Sonuçları Görselleştirme
def visualize_results(store_results):
    """
    Sonuçları görselleştir
    """
    for store, result in store_results.items():
        plt.figure(figsize=(14, 8))
        
        train_data = result['train_data']
        predictions = result['predictions']
        actual_values = result['actual_values']
        
        # Eğitim verilerini çiz
        plt.plot(train_data['Ay'], train_data['Ciro'], 
                label='Eğitim Verisi (2023-2024)', marker='o', linewidth=2)
        
        # Tahminleri çiz
        pred_dates = [pd.Timestamp('2025-01-31')]
        pred_values = [predictions['jan']]
        plt.plot(pred_dates, pred_values, 'ro-', 
                label='Tahminler', markersize=8, linewidth=2)
        
        # Gerçek değerleri çiz (varsa)
        if actual_values:
            real_dates = [pd.Timestamp('2025-01-31')]
            real_values = [actual_values.get(1, np.nan)]
            plt.plot(real_dates, real_values, 'go-', 
                    label='Gerçek Değerler', markersize=8, linewidth=2)
        
        plt.title(f'Mağaza {store} - Ciro Tahmini (Çok Değişkenli Model)', fontsize=14)
        plt.xlabel('Tarih', fontsize=12)
        plt.ylabel('Ciro (TL)', fontsize=12)
        plt.legend(fontsize=11)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Özet istatistikler
        print(f"\nMağaza {store} Özet:")
        print(f"Kullanılan özellik sayısı: {len(result['selected_features'])}")
        print(f"Seçilen özellikler: {result['selected_features']}")
        if actual_values and 1 in actual_values:
            mae = abs(predictions['jan'] - actual_values[1])
            mape = abs(predictions['jan'] - actual_values[1]) / actual_values[1] * 100
            print(f"Ocak 2025 MAE: {mae:.2f} TL")
            print(f"Ocak 2025 MAPE: {mape:.2f}%")
        print("-" * 50)

# 8. Ana çalıştırma fonksiyonu
if __name__ == "__main__":
    # Modeli eğit ve tahmin yap
    results = train_and_predict()
    
    # Sonuçları görselleştir
    visualize_results(results)